Library

In [ ]:
# Importing Library
import math
import pandas as pd

Training

In [ ]:
# Kelompokkan setiap baris data sesuai kelas output 1 atau 0
# menggunakan dictionary untuk setiap baris data 
# Output dijadikan key, Setiap baris data dijadikan value
# ex group = {1: ([60, 64, 0, 1], ...), 0: ([54, 60, 11, 0], ...)}
def groupByKelas(data):
      group = {}
      for i in range(len(data)):
          if (data[i][-1] not in group):
              group[data[i][-1]] = []
          group[data[i][-1]].append(data[i])
      return group

# Hitung Mean
def mean(list_angka):
    return sum(list_angka) / float(len(list_angka))
  
# Hitung Standar Deviasi atau Simpangan Baku
def std_dev(list_angka):
    rata2 = mean(list_angka)
    # Varians = (Standar Deviasi)^2 -> Standar Devisasi = Akar(Varians)
    varians = sum([pow(angka - rata2, 2) for angka in list_angka]) / float(len(list_angka) - 1)
    return math.sqrt(varians)

def MeanAndStdDev(data):
    info = [(mean(atribut), std_dev(atribut)) for atribut in zip(*data)]
    # ex: list = [ [a, b, c,..], [m, n, o,..], [x, y, z,..]]
    # ex Mean: Untuk atribut pertama =(a+m+x)/3, mean of atribut kedua = (b+n+y)/3, dst..
    # Delete Summaries Untuk Index Terakhir Karena Merupakan Output
    del info[-1]
    return info

# Hitung Mean and Standard Deviasi Setiap Atribut Berdasarkan output
# Untuk Membuat Model Training
def createModel(data):
    # Model adalah Hasil dari Training Data 
    # Isi Model adalah Mean dan Std Deviasi setiap atribut berdasarkan output 0 atau 1
    model = {}
    group = groupByKelas(data)
    for output, data in group.items():
        model[output] = MeanAndStdDev(data)
    return model


Testing

In [ ]:
# Hitung Peluang Gaussian atau Distribusi Normal
def hitungProbabilitasGaussian(x, mean, stdev):
    exponen = math.exp(-(math.pow(x - mean, 2) / (2 * math.pow(stdev, 2))))
    return (1 / (math.sqrt(2 * math.pi) * stdev)) * exponen

# Hitung Probabilitas Setiap Kelas Output 0 dan 1
def hitungProbabilitasKelas(model, test):
    probabilitas = {}
    for output, classSummaries in model.items():
        probabilitas[output] = 1
        # Perhitungan Peluang Gaussian
        for i in range(len(classSummaries)):
            mean, std_dev = classSummaries[i]
            x = test[i]
            probabilitas[output] *= hitungProbabilitasGaussian(x, mean, std_dev)
    return probabilitas

# Membuat Prediksi 
def prediksi(model, test):
    probabilitas = hitungProbabilitasKelas(model, test)
    bestLabel = None
    bestProb = -1
    for output, prob in probabilitas.items():
        # Probabilitas Tertinggi Menjadi Prediksi
        if bestLabel is None or prob > bestProb:
            bestProb = prob
            bestLabel = output
    return bestLabel

# Prediksi Output Untuk Data Testing
def prediksiOutput(model, test):
    prediksiOutput = []
    for i in range(len(test)):
        result = prediksi(model, test[i])
        prediksiOutput.append(result)
    return prediksiOutput

Evaluasi

In [ ]:
# Evaluasi - Cek Akurasi
def tesAkurasi(test, prediksiOutput):
    correct = 0
    for i in range(len(test)):
        if test[i][-1] == prediksiOutput[i]:
            correct += 1
    return (correct / float(len(test))) * 100.0

Main Program / Driver Code

In [ ]:
# Main Program

# Read Excel File
data_file = pd.ExcelFile(r'traintest.xlsx')

# Training
# Load Excel File Untuk Data Training
train_file = pd.read_excel(data_file, 'train')
# Taruh setiap baris data training pada list train_data
train_data = pd.DataFrame(train_file, columns = ['x1', 'x2', 'x3', 'y'])
train_data = train_data.values
# Ubah setiap value data menjadi bilangan real
for i in range(len(train_data)):
    train_data[i] = [float(data) for data in train_data[i]]
# Simpan Model Hasil Training
model = createModel(train_data)
# Print Model Training
print("=======================Hasil Training========================")
for output, mean_stdDev in model.items():
  for i in range(3):
    print("(Mean, StdDev) x{} C{}: {}".format(i+1, output, mean_stdDev[i]))
print("=======================Evaluasi Model========================")

# Evaluasi Model
eval_data = train_data
hasil = prediksiOutput(model, eval_data)
akurasi = tesAkurasi(eval_data, hasil)
print("Akurasi Model Training: ", akurasi)

# Testing
# Load Excel File Untuk Data Testing
test_file = pd.read_excel(data_file, 'test')
# Taruh setiap baris data testing pada list test_data
test_data = pd.DataFrame(test_file, columns = ['x1', 'x2', 'x3', 'y'])
# Ganti value pada kolom y dari '?' menjadi -1 agar tidak error
test_data['y'].replace({'?' : -1}, inplace=True)
test_data = test_data.values
# Ubah setiap value data menjadi bilangan real
for i in range(len(test_data)):
    test_data[i] = [float(data) for data in test_data[i]]

print("=========================Info Data===========================")
# Total Keseluruhan Data
print('Data Total   : ', len(train_data) + len(test_data))
# Total Data Training
print('Data Training: ', len(train_data))
# Total Data Testing
print("Data Testing : ", len(test_data))

# Hasil Testing Model
hasil = prediksiOutput(model, test_data)
print("========================Hasil Testing========================")
print("Hasil Prediksi: ")
test_data = pd.DataFrame(test_file, columns = ['x1', 'x2', 'x3', 'y'])
test_data = test_data.values
for i in range(len(test_data)):
    print("[{}]".format(i+1), test_data[i], ": ", hasil[i])
print("========================Output File=========================")

# Simpan Output ke File
result = {}
id_dan_atribut = pd.DataFrame(test_file, columns = ['id', 'x1', 'x2', 'x3'])
id_dan_atribut.values
result['id'] = id_dan_atribut['id']
result['x1'] = id_dan_atribut['x1']
result['x2'] = id_dan_atribut['x2']
result['x3'] = id_dan_atribut['x3']
result['y'] = hasil
result = pd.DataFrame(result, columns = ['id', 'x1', 'x2', 'x3', 'y'])
print(result)
result.to_excel('result.xlsx', sheet_name = 'result', index = False)

=======================Hasil Training========================
(Mean, StdDev) x1 C1: (51.93577981651376, 11.110484299554663)
(Mean, StdDev) x2 C1: (62.92660550458716, 3.222062258631095)
(Mean, StdDev) x3 C1: (2.8394495412844036, 5.953499042384458)
(Mean, StdDev) x1 C0: (53.93589743589744, 10.198471675149396)
(Mean, StdDev) x2 C0: (62.756410256410255, 3.2839076241650407)
(Mean, StdDev) x3 C0: (7.666666666666667, 9.296929623961898)
=======================Evaluasi Model========================
Akurasi Model Training:  76.35135135135135
=========================Info Data===========================
Data Total   :  306
Data Training:  296
Data Testing :  10
========================Hasil Testing========================
Hasil Prediksi: 
[1] [43 59 2 '?'] :  1
[2] [67 66 0 '?'] :  1
[3] [58 60 3 '?'] :  1
[4] [49 63 3 '?'] :  1
[5] [45 60 0 '?'] :  1
[6] [54 58 1 '?'] :  1
[7] [56 66 3 '?'] :  1
[8] [42 69 1 '?'] :  1
[9] [50 59 2 '?'] :  1
[10] [59 60 0 '?'] :  1
========================Output 